In [1]:
import os
import torch
from torchvision import datasets, transforms
from PIL import Image, ImageOps

In [ ]:
# train the model with `https://github.com/kefth/fashion-mnist`

In [2]:
train_transforms = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.1307,), (0.3081,))
])
val_transforms = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.1307,), (0.3081,))
])

fashion_mnist_data_path = "/root/code/fashion-mnist/data"
trainset = datasets.FashionMNIST(fashion_mnist_data_path, train=True, download=True, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(trainset, shuffle=True)
valset = datasets.FashionMNIST(fashion_mnist_data_path, train=False, transform=val_transforms)
val_loader = torch.utils.data.DataLoader(valset, shuffle=True)

In [3]:
trainset.data.shape, trainset.data.dtype

(torch.Size([60000, 28, 28]), torch.uint8)

In [14]:
for X, y in train_loader:
  print(X.shape, X.dtype)
  print(y.shape, y.dtype)
  break

torch.Size([1, 1, 28, 28]) torch.float32
torch.Size([1]) torch.int64


In [5]:
classes = trainset.classes
class_to_idx = trainset.class_to_idx
idx_to_class = {v: k for k, v in trainset.class_to_idx.items()}

In [6]:
# taken from `https://github.com/kefth/fashion-mnist`
import torch.nn as nn

class FashionSimpleNet(nn.Module):

    """ Simple network"""

    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1,32, kernel_size=3, padding=1), # 28
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 14
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2) # 7
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 64 * 7 * 7)
        x = self.classifier(x)
        return x

In [7]:
moodel_path = "/root/code/ml-scaffold/kefth/saved-models/FashionSimpleNet-run-1.pth.tar"

net = FashionSimpleNet()
_ = net.load_state_dict(torch.load(moodel_path, weights_only=True)["state_dict"])
net.eval()  # Set the model to evaluation mode

FashionSimpleNet(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [8]:
def predict_img(net, img_path):
  print("img path:", img_path)
  test_img = Image.open(img_path)
  test_img = test_img.convert("L")
  #test_img.show()
  #test_img.mode, test_img.size, test_img.format
  fit_img = ImageOps.fit(test_img, (28, 28))
  fit_img.show()
  img_tensor = train_transforms(fit_img)
  output = net(img_tensor.unsqueeze(0))
  #print("Output:", output)
  predicted_label = output.softmax(1).argmax(1).item()
  print("Predicted label:", predicted_label, "Predicted class:", idx_to_class[predicted_label])

In [9]:
test_img_path = "/root/code/fashion-mnist/test-imgs"
for image in os.listdir(test_img_path):
    img_path = os.path.join(test_img_path, image)
    predict_img(net, img_path)

img path: /root/code/fashion-mnist/test-imgs/sneakers.jpeg


Predicted label: 8 Predicted class: Bag
img path: /root/code/fashion-mnist/test-imgs/trouser.jpeg


Predicted label: 8 Predicted class: Bag
img path: /root/code/fashion-mnist/test-imgs/sandal.jpeg


Predicted label: 8 Predicted class: Bag
img path: /root/code/fashion-mnist/test-imgs/t-shirt.jpeg


Predicted label: 8 Predicted class: Bag


In [10]:
to_pil = transforms.ToPILImage()

print("test on training dataset")

total_count = 0
err_count = 0
for i, (X, y) in enumerate(train_loader):
  total_count += 1
  output = net(X)
  y_hat = output.softmax(1).argmax(1).item()
  y = y.item()
  if y_hat != y:
    #print("predict: {}, actual: {}".format(idx_to_class[y_hat], idx_to_class[y]))
    #to_pil(X.squeeze()).show()
    err_count += 1

print("total_count: {}, err_count: {}, accuracy: {:.2f}%".format(total_count, err_count, (total_count-err_count)*100/total_count))

test on training dataset


total_count: 60000, err_count: 1569, accuracy: 97.39%


In [11]:
print("test on validation dataset")

total_count = 0
err_count = 0
for i, (X, y) in enumerate(val_loader):
  total_count += 1
  output = net(X)
  y_hat = output.softmax(1).argmax(1).item()
  y = y.item()
  if y_hat != y:
    #print("predict: {}, actual: {}".format(idx_to_class[y_hat], idx_to_class[y]))
    #to_pil(X.squeeze()).show()
    err_count += 1

print("total_count: {}, err_count: {}, accuracy: {:.2f}%".format(total_count, err_count, (total_count-err_count)*100/total_count))

test on validation dataset
total_count: 10000, err_count: 689, accuracy: 93.11%


In [12]:
onnx_model_path = "/root/code/ml-scaffold/kefth/saved-models/model.onnx"
torch.onnx.export(
    net,                      # Model to be exported
    X,                        # Dummy input tensor
    onnx_model_path,             # Output file path
    export_params=True,         # Store the trained parameter weights inside the model file
    #opset_version=11,           # ONNX version to export the model to
    do_constant_folding=True,   # Whether to execute constant folding for optimization
    input_names=['input'],      # Input tensor names
    output_names=['output'],    # Output tensor names
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # Dynamic axes
)


In [13]:
import onnx

# Load the ONNX model
onnx_model = onnx.load(onnx_model_path)
# Check that the model is well-formed
onnx.checker.check_model(onnx_model)
print("The model has been successfully converted to ONNX format.")

The model has been successfully converted to ONNX format.


In [ ]:
# setup model repository for serving

'''
# tree                   
.
`-- kefth-fashion-mnist
    `-- 1
        |-- config.pbtxt
        |-- model.onnx
        `-- model.py
'''

In [26]:
import numpy as np
import tritonclient.grpc as grpcclient
from tritonclient.utils import np_to_triton_dtype, triton_to_np_dtype
from tritonclient.utils import InferenceServerException

server_addr = "localhost:8001"
triton_client = grpcclient.InferenceServerClient(server_addr)

in0 = X.numpy()
input_tensors = [
    grpcclient.InferInput("input", in0.shape, np_to_triton_dtype(in0.dtype)).set_data_from_numpy(in0),
]
output_tensors = [
    grpcclient.InferRequestedOutput("output"),
]
model_name = "kefth-fashion-mnist"
infer_rsp = triton_client.infer(model_name, inputs=input_tensors, outputs=output_tensors)

In [27]:
output0 = infer_rsp.as_numpy("output")
print(output0)

[[-6.09344   -2.4736507 -4.753368   5.8362384 -5.3823166 -3.6607287
  -8.433782  -4.2557     0.6277533 -7.025732 ]]
